Amazon Bedrock AgentCore Gateway - Semantic search tutorial

In [2]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

In [3]:
# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

In [4]:
boto3.__version__

'1.42.1'

In [5]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
)

Creating Amazon Cognito resources

In [6]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

[
    {
        "name": "create_booking",
        "description": "Create a new booking at restaurant_name",
        "inputSchema": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "The date of the booking in the format YYYY-MM-DD. Do NOT accept relative dates like today or tomorrow. Ask for today's date for relative date."
                },
                "hour": {
                    "type": "string",
                    "description": "the hour of the booking in the format HH:MM"
                },
                "restaurant_name": {
                    "type": "string",
                    "description": "name of the restaurant handling the reservation"
                },
                "guest_name": {
                    "type": "string",
                    "description": "The name of the customer to have in the reservation"
                },
                "num_gues

In [7]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

[
    {
        "name": "add_numbers",
        "description": "Adds firstNumber and secondNumber together to get the sum (firstNumber + secondNumber)",
        "inputSchema": {
            "type": "object",
            "properties": {
                "firstNumber": {
                    "type": "number",
                    "description": "first number to add"
                },
                "secondNumber": {
                    "type": "number",
                    "description": "second number to add"
                }
            },
            "required": [
                "firstNumber",
                "secondNumber"
            ]
        }
    },
    {
        "name": "subtract_numbers",
        "description": "Subtracts the subtrahend from the minuend to find the difference (minuend - subtrahend)",
        "inputSchema": {
            "type": "object",
            "properties": {
                "minuend": {
                    "type": "number",
                    "descripti

In [8]:
# AWS Lambda function implementation for the calculator tools.
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

def get_named_parameter(event, name):
    return event[name]


def handle_add(event):
    firstNumber = int(get_named_parameter(event, "firstNumber"))
    secondNumber = int(get_named_parameter(event, "secondNumber"))
    return {"sum": firstNumber + secondNumber}


def handle_multiply(event):
    multiplicand = int(get_named_parameter(event, "multiplicand"))
    multiplier = int(get_named_parameter(event, "multiplier"))
    return {"product": multiplicand * multiplier}


def handle_divide(event):
    divisor = int(get_named_parameter(event, "divisor"))
    dividend = int(get_named_parameter(event, "dividend"))

    if divisor == 0:
        raise Exception("Divisor cannot be 0")

    quotient = dividend / divisor

    return {"quotient": quotient}


def handle_subtract(event):
    minuend = int(get_named_parameter(event, "minuend"))
    subtrahend = int(get_named_parameter(event, "subtrahend"))

    difference = minuend - subtrahend

    return {"difference": difference}


def lambda_handler(event, context):
    print(f"event: {event}")
    print(f"context: {context}")
    print(f"context.client_context: {context.client_context}")

    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    tool_name = extended_tool_name.split("___")[1]

    print(f"tool_name: {tool_name}")

    if tool_name == "add_numbers":
        result = handle_add(event)
    elif tool_name == "multiply_numbers":
        result = handle_multiply(event)
    elif tool_name == "divide_numbers":
        result = handle_divide(event)
    elif tool_name == "subtract_numbers":
        result = handle_subtract(event)
    else:
        result = f"Unrecognized tool_name: {tool_name}"

    print(f"result: {result}")
    return result

In [9]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

def get_named_parameter(event, name):
    return event[name]


def handle_create_booking(event):
    bookingDate = get_named_parameter(event, "date")
    bookingHour = get_named_parameter(event, "hour")
    restaurantName = get_named_parameter(event, "restaurant_name")
    guestName = get_named_parameter(event, "guest_name")
    numGuests = int(get_named_parameter(event, "num_guests"))
    return f"Booking id 12345, for {numGuests} guests at {restaurantName} on {bookingDate} at {bookingHour} for {guestName} created."


def lambda_handler(event, context):
    print(f"event: {event}")
    print(f"context: {context}")
    print(f"context.client_context: {context.client_context}")

    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    tool_name = extended_tool_name.split("___")[1]

    print(f"tool_name: {tool_name}")

    if tool_name == "create_booking":
        result = handle_create_booking(event)
    else:
        result = f"Unrecognized tool_name: {tool_name}"

    print(f"result: {result}")
    return result

In [10]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            calc_lambda_resp["lambda_function_arn"],
        )

Reading code from zip file
Creating IAM role for lambda function
Attaching policy to the IAM role
Role 'calc_lambda_gateway_lambda_iamrole' created successfully: arn:aws:iam::381491838394:role/calc_lambda_gateway_lambda_iamrole
Creating lambda function
Lambda function created with ARN:  arn:aws:lambda:ap-southeast-2:381491838394:function:calc_lambda_gateway


In [11]:
calc_lambda_resp["lambda_function_arn"]

'arn:aws:lambda:ap-southeast-2:381491838394:function:calc_lambda_gateway'

In [12]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

Reading code from zip file
Creating IAM role for lambda function
Attaching policy to the IAM role
Role 'restaurant_lambda_gateway_lambda_iamrole' created successfully: arn:aws:iam::381491838394:role/restaurant_lambda_gateway_lambda_iamrole
Creating lambda function
Lambda function created with ARN:  arn:aws:lambda:ap-southeast-2:381491838394:function:restaurant_lambda_gateway


In [13]:
restaurant_lambda_resp["lambda_function_arn"]

'arn:aws:lambda:ap-southeast-2:381491838394:function:restaurant_lambda_gateway'

In [14]:
cognito_response = utils.setup_cognito_user_pool()

Creating Cognito User Pool: MCPServerPool
User Pool created with ID: ap-southeast-2_EIdoHjSsM
Creating Cognito App Client: MCPServerPoolClient
App Client created with ID: ivfma8uds7je9qqgvjfk01rpj
Creating Cognito user: testuser
Setting permanent password for user: testuser
Pool ID: ap-southeast-2_EIdoHjSsM
Discovery URL: https://cognito-idp.ap-southeast-2.amazonaws.com/ap-southeast-2_EIdoHjSsM/.well-known/openid-configuration
Client ID: ivfma8uds7je9qqgvjfk01rpj


In [15]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

Authenticating user: testuser
Bearer token obtained successfully


In [16]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

Creating IAM role: GatewaySearchAgentCoreRole
Attaching policy: BedrockAgentPolicy
Gateway IAM role created successfully: arn:aws:iam::381491838394:role/GatewaySearchAgentCoreRole


In [17]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string
        with open(json_file_path, "r") as file:
            # Parse JSON to Python object
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

In [19]:
#  Create Gateway helper function
def create_gateway(gateway_name, gateway_desc):
    # Use Cognito for Inbound OAuth to our Gateway
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # Enable semantic search of tools
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Create the gateway
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    # print(json.dumps(response, indent=2, default=str))
    return response["gatewayId"]

In [20]:
# Create Gateway Target helper function
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Add a Lambda target to the gateway
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # Use IAM as credential provider
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

Creating your first AgentCore Gateway

In [21]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

Create gateway with name: gateway-search-tutorial
Gateway created with id: gateway-search-tutorial-mgbqigzswr.


Adding AgentCore Gateway Targets

In [22]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

Restaurant Lambda ARN: arn:aws:lambda:ap-southeast-2:381491838394:function:restaurant_lambda_gateway
RestaurantTarget created with id: FOPRSSIUSK on gateway: gateway-search-tutorial-mgbqigzswr


In [23]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

API spec for calc has 79 functions

Calc Lambda ARN: arn:aws:lambda:ap-southeast-2:381491838394:function:calc_lambda_gateway
CalcTools Target created with id: GCIA0OWXYT on gateway: gateway-search-tutorial-mgbqigzswr


To demonstrate the power of gateway search, now we add a few more copies of the Calculator target, so that we end up with 300+ MCP tools exposed.

In [24]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [25]:
add_more_tools(gatewayId=gatewayId)

Calc2 Target created with id: DYS7MOFRL4 on gateway: gateway-search-tutorial-mgbqigzswr
Calc3 Target created with id: ZLKTJQLHG4 on gateway: gateway-search-tutorial-mgbqigzswr
Calc4 Target created with id: SQSNDDRBB2 on gateway: gateway-search-tutorial-mgbqigzswr


In [26]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

Calc2 - Calculation 2 Tools
FoodTools - Restaurant Tools
CalcTools - Calculation Tools
Calc4 - Calculation 4 Tools
Calc3 - Calculation 3 Tools


Searching for tools from a Gateway

In [27]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

In [28]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

Gateway Endpoint - MCP URL: https://gateway-search-tutorial-mgbqigzswr.gateway.bedrock-agentcore.ap-southeast-2.amazonaws.com/mcp


In [29]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

Authenticating user: testuser
Bearer token obtained successfully
Bearer token: eyJraWQiOiJqZTExcW1wTHU3XC9KamhvSXREV0J4M0p4U3RZRW9CTVhibVY0ZVZOWWtCdz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI4OWRlMTQwOC0zMDIxLTcwNzEtOGQ2MS1hMTk1MjNjOTY4MzYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtc291dGhlYXN0LTIuYW1hem9uYXdzLmNvbVwvYXAtc291dGhlYXN0LTJfRUlkb0hqU3NNIiwiY2xpZW50X2lkIjoiaXZmbWE4dWRzN2plOXFxZ3ZqZmswMXJwaiIsIm9yaWdpbl9qdGkiOiJmNTM4YTYzNi05MjA2LTRlMzQtOTgxNS1iMTUyMTJjYmRmZmMiLCJldmVudF9pZCI6ImJjNDAxMTRlLTRjOWYtNDQzZi1iODM1LTZjZjA2ZDJkZjczNSIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjQ3MTY2NjIsImV4cCI6MTc2NDcyMDI2MiwiaWF0IjoxNzY0NzE2NjYyLCJqdGkiOiJlZTJiNTM0OC01Mjc2LTRlZDctODJkNS04NmFmNjY2MGFkNjEiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.bUSGocuYTXoIfUv1NedencRzAgN9l9nnyTp25j8kdSRS2iiJQgnSyUEOOovVEYlXyz8atq15xCVXKVwrnKCCPmGrknPLI24UYldH1ETrseE3CB24QgscVi2-OsxS3oYU33obwbXnXbPly2_DHDRu_kz-R4l2adrUyrBmSvxoPCb-qFpuKneyq82QeByZ-EaVduBtCIkUkmSAfb3yLBb7sU8n

In [30]:
# !npx @modelcontextprotocol/inspector

In [31]:
# Creating helper functions that use jsonrpc to invoke MCP tools or list them
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"Invoking tool {tool_params['name']}")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

In [32]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

In [33]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")



Listing tools for gateway https://gateway-search-tutorial-mgbqigzswr.gateway.bedrock-agentcore.ap-southeast-2.amazonaws.com/mcp
adding tool 'x_amz_bedrock_agentcore_search' - A special tool that returns a trimmed do...
adding tool 'Calc2___add_numbers' - Adds firstNumber and secondNumber togeth...
adding tool 'Calc2___analyze_bond_liquidity' - Analyzes liquidity metrics for fixed inc...
adding tool 'Calc2___analyze_covenant_compliance' - Analyzes compliance with bond covenants ...
adding tool 'Calc2___analyze_debt_structure' - Analyzes the debt structure of an issuer...
adding tool 'Calc2___analyze_fund_flows' - Analyzes fund inflows and outflows over ...
adding tool 'Calc2___analyze_peer_comparison' - Compares a portfolio or fund against pee...
adding tool 'Calc2___analyze_portfolio_liquidity' - Analyzes the liquidity profile of a port...
adding tool 'Calc2___analyze_technical_indicators' - Calculates technical indicators for a se...
adding tool 'Calc2___analyze_time_series' - Perfo

In [34]:
# Using Strands Agents list_tools_sync() with pagination
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

In [35]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")


Found 318 tools from list_tools_sync() on mcp client



Using the built-in Gateway semantic search tool

In [37]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [39]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

# Notice how fast the search returns, in under a second in most cases.
# The results are returned in descending order of search relevance based on matching
# the query to the tool metadata. The most relevant tools are first on the list.
# The intial implementation of search gives back up to 10 results. You could then
# use all of these tools in your agent, or simply pick a subset of the most relevant matches.

tool search via direct Gateway invocation took 0.70 seconds
Top tool: CalcTools___get_credit_rating_history


Using Strands Agents with an MCP server that has many tools

In [40]:
bedrockmodel = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

In [41]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")

Authenticating user: testuser
Bearer token obtained successfully

Found 318 tools from list_tools_sync() on mcp client

100 plus 50 equals **150**.


In [43]:
# The Strands Agents framework also lets you bypass the agent event loop,
# invoking an MCP tool directly. Since Gateway tools
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

Authenticating user: testuser
Bearer token obtained successfully

Found 318 tools from list_tools_sync() on mcp client

direct result = {'status': 'success', 'toolUseId': 'tooluse_Calc2___add_numbers_186744919', 'content': [{'text': '{"sum":30}'}]}


In [44]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [45]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))
    return search_results

In [46]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [47]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

Authenticating user: testuser
Bearer token obtained successfully
[
    {
        "inputSchema": {
            "type": "object",
            "properties": {
                "technicalCriteria": {
                    "type": "object",
                    "properties": {
                        "rsiMax": {
                            "type": "number"
                        },
                        "rsiMin": {
                            "type": "number"
                        },
                        "priceAboveSMA50": {
                            "type": "boolean"
                        },
                        "volumeIncreasePercent": {
                            "type": "number"
                        }
                    }
                },
                "universe": {
                    "type": "string"
                },
                "fundamentalCriteria": {
                    "type": "object",
                    "properties": {
                        "peRatioM

In [48]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

Authenticating user: testuser
Bearer token obtained successfully
{
    "inputSchema": {
        "type": "object",
        "properties": {
            "technicalCriteria": {
                "type": "object",
                "properties": {
                    "rsiMax": {
                        "type": "number"
                    },
                    "rsiMin": {
                        "type": "number"
                    },
                    "priceAboveSMA50": {
                        "type": "boolean"
                    },
                    "volumeIncreasePercent": {
                        "type": "number"
                    }
                }
            },
            "universe": {
                "type": "string"
            },
            "fundamentalCriteria": {
                "type": "object",
                "properties": {
                    "peRatioMax": {
                        "type": "number"
                    },
                    "debtToEquityMax": {
  

Adding tool search results to a Strands Agent

In [49]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [50]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")

Authenticating user: testuser
Bearer token obtained successfully
I'll calculate this step by step.

First, let me calculate the numerator (10*2) and the denominator (5-3):
Tool #1: CalcTools___multiply_numbers

Tool #2: CalcTools___subtract_numbers


WARNING | client | Tool CalcTools___multiply_numbers not listed by server, cannot validate any structured content
WARNING | client | Tool CalcTools___subtract_numbers not listed by server, cannot validate any structured content


Now let me divide the numerator by the denominator (20 ÷ 2):
Tool #3: CalcTools___divide_numbers


WARNING | client | Tool CalcTools___divide_numbers not listed by server, cannot validate any structured content


**(10*2)/(5-3) = 10****(10*2)/(5-3) = 10**


In [ ]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")

    #  Notice the latency improvement. This example using a subset of tools 
    # from Gateway search is significantly faster than agent invocation
    # when depending on hundreds of tools.

Authenticating user: testuser
Bearer token obtained successfully
Searching for an ADDING tool from endpoint with full set of tools...
Top tool found: CalcTools___multiply_numbers


Tool #1: CalcTools___multiply_numbers


WARNING | client | Tool CalcTools___multiply_numbers not listed by server, cannot validate any structured content


10 × 70 = **700**10 × 70 = **700**
CPU times: user 235 ms, sys: 20.6 ms, total: 256 ms
Wall time: 4.87 s


Showing 3x latency improvement by using tool search

In [52]:
#  Measure latency and token usage for agent using the entire set of MCP tools
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

Authenticating user: testuser
Bearer token obtained successfully


In [53]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")


Found 318 tools from list_tools_sync() on mcp client

using agent with ALL tools...
1) 100 plus 1 equals **101**.
1) I'd be happy to help you book a table at Burger King! However, I need to know the year for August 1st. Could you please provide the year (for example, 2024 or 2025)?
2) 100 plus 2 equals **102**.
2) I'd be happy to help you book a table at Burger King! However, I need to know the year for August 2nd. Could you please provide the year (for example, 2024 or 2025)?

Total time: 23.2 s, tokens: 303,273



In [54]:
#  Measure latency and token usage for agents using Gateway Search
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

using agent with ONLY tools from focused search...
Searching for an ADDING tool from endpoint with full set of tools...
Top tool found: CalcTools___add_numbers



WARNING | client | Tool CalcTools___add_numbers not listed by server, cannot validate any structured content


1) 100 plus 1 equals **101**.
Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools...
Top tool found: FoodTools___create_booking

1) I'd be happy to help you book a table at Burger King! However, I need to know the year for August 1st. Could you please provide the year (for example, 2024 or 2025)?
Searching for an ADDING tool from endpoint with full set of tools...
Top tool found: CalcTools___add_numbers



WARNING | client | Tool CalcTools___add_numbers not listed by server, cannot validate any structured content


2) 100 plus 2 equals **102**.
Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools...
Top tool found: FoodTools___create_booking

2) I'd be happy to help you book a table at Burger King! However, I need to know the year for August 2nd. Could you please provide the year (for example, 2024 or 2025)?

Total time: 60.2 s, tokens: 1,104



In [55]:
# Compare results, higlighting benefits of search
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")



Latency without search: 23.2s, using search: 60.2s
Tokens without search: 303,273, using search: 1,104


Cleaning up resources

In [ ]:
# def delete_gatewaytarget(gateway_id):
#     response = agentcore_client.list_gateway_targets(gatewayIdentifier=gateway_id)

#     print(f"Found {len(response['items'])} targets for the gateway")

#     for target in response["items"]:
#         print(
#             f"Deleting target with Name: {target['name']} and Id: {target['targetId']}"
#         )

#         response = agentcore_client.delete_gateway_target(
#             gatewayIdentifier=gateway_id, targetId=target["targetId"]
#         )
#         time.sleep(20)


# def delete_gateway(gateway_id):
#     response = agentcore_client.delete_gateway(gatewayIdentifier=gateway_id)

In [57]:
# delete_gatewaytarget(gateway_id=gatewayId)

In [58]:
# lambda_arns = [
#     calc_lambda_resp["lambda_function_arn"],
#     restaurant_lambda_resp["lambda_function_arn"],
# ]

# for arn in lambda_arns:
#     if utils.delete_gateway_lambda(arn):
#         print(f"Deleted Lambda: {arn}")
#     else:
#         print(f"Lambda {arn} not found or deletion failed")

In [ ]:
# # Gateway role cleanup
# if utils.delete_gateway_iam_role():
#     print("Gateway IAM role deleted")
# else:
#     print("Gateway IAM role not found or deletion failed")

# # Cognito cleanup
# if utils.delete_cognito_user_pool():
#     print("Cognito pool deleted")
# else:
#     print("✗ Failed to delete Cognito pool")